This EDA nb is using occurrence tables e-mailed from Tiago on 9/27 (duplicated, with rotated lat and long columns)

## Imports + Loading In Data

In [6]:
import pandas as pd


In [7]:
%pip install openpyxl

In [8]:
# Loading in data w/ rotated coordinates
rep = pd.read_excel('reptilia_occ_genus.xlsm', engine='openpyxl')
syn = pd.read_excel('synapsida_occ_genus.xlsm', engine='openpyxl')
tem = pd.read_excel('temnospondyli_occ_genus.xlsm', engine='openpyxl')

In [9]:
rep['county'].isna().sum()

1495

In [10]:
rep['county'].value_counts()

Apache                    234
Comanche                  152
Rio Arriba                 88
Franklin                   78
Gloucestershire            67
                         ... 
Malmyzh                     1
Anson                       1
Sa√É¬¥ne-et-Loire           1
Huesca                      1
Aliwal North Commonage      1
Name: county, Length: 355, dtype: int64

## Saving Nulls as CSV's

In [11]:
# Seeing how many nulls are in each column will help us decide which columns to drop
nulls_rep = rep.isna().sum()
nulls_syn = syn.isna().sum()
nulls_tem = tem.isna().sum()

# Concatenating the nulls into a single dataframe based on a shared index
# This will allow us to compare the nulls across the three datasets
nulls = pd.concat([nulls_rep, nulls_syn, nulls_tem], axis=1)
nulls.columns = ['Reptilia', 'Synapsida', 'Temnospondyli']

nulls.to_csv('nulls.csv')

In [12]:
# len of nulls df is diff from len of columns in rep
# which means there are columns that are not shared
len(nulls.index), len(rep.columns)

(119, 94)

In [13]:
# Finding which columns rep, syn, and tem do not share
# Crude code, just to see if there are any differences, not finding all differences
# Because any not-shared columns wouldn't be included in the analysis anyway

if rep.columns.all() == syn.columns.all() == tem.columns.all():
    print('All columns are the same')
else:
    diff = rep.columns.difference(syn.columns).difference(tem.columns)
    print(diff)

Index(['abund_in_sediment', 'artifacts', 'component_comments', 'concentration',
       'fossilsfrom1', 'fossilsfrom2', 'lagerstatten', 'lithadj2', 'localbed',
       'localsection', 'minor_lithology2', 'orientation', 'reference_no.x',
       'temporal_resolution'],
      dtype='object')


## Confirming that these are Datasets w/ duplicated rows (eda_duplicates)

In [14]:
# Loading in datasets that I had previously duplicated (i.e., the output of eda_duplicates.ipynb)
rep_old = pd.read_csv('occurrence_tables_1/reptilia_dup.csv')
syn_old = pd.read_csv('occurrence_tables_1/synapsida_dup.csv')
tem_old = pd.read_csv('occurrence_tables_1/temnospondyli_dup.csv')

In [15]:
# Checking that the lengths of the original and new datasets are the same
len(rep_old), len(rep), len(syn_old), len(syn), len(tem_old), len(tem)

(4411, 4411, 3422, 3422, 2247, 2247)

In [16]:
# One last double-check to make sure that the datasets are the same
# All 'abund_values' for "individuals" should be 1
individuals = rep[rep['abund_unit'] == 'individuals']
individuals['abund_value'].value_counts()

1.0    2032
Name: abund_value, dtype: int64

## Filtering to BDNN columns

In [17]:
rep.columns

Index(['id', 'Rotated Lat', 'Rotated Lon', 'occurrence_no', 'pres_mode',
       'preservation_quality', 'common_body_parts', 'abund_value',
       'abund_unit', 'class', 'family', 'genus', 'taxon_environment',
       'life_habit', 'diet', 'early_interval', 'late_interval', 'time_bins',
       'museum', 'collection_no', 'age_max', 'age_min', 'age_median',
       'age_uncer_range', 'lng', 'lat', 'environment', 'occurrence_comments',
       'cc', 'state', 'county', 'formation', 'stratgroup', 'member', 'zone',
       'accepted_rank', 'ref_author', 'ref_pubyr', 'reference_no.x',
       'collection_name', 'collection_subset', 'collection_aka',
       'latlng_basis', 'latlng_precision', 'geogscale', 'geogcomments',
       'paleomodel', 'geoplate', 'paleoage', 'paleolng', 'paleolat',
       'paleomodel2', 'geoplate2', 'paleoage2', 'paleolng2', 'paleolat2',
       'paleomodel3', 'geoplate3', 'paleoage3', 'paleolng3', 'paleolat3',
       'protected', 'stratscale', 'localsection', 'localbed', 'st

In [89]:
# Still waiting on 'niche' info from Arielli
syn_filtered = syn[['Rotated Lat', 'Rotated Lon', 'genus', 'taxon_environment', 
                    'age_max', 'age_min']]
syn_filtered.isna().sum()

Rotated Lat            0
Rotated Lon            0
genus                  0
taxon_environment    310
age_max                0
age_min                0
dtype: int64

In [90]:
# Still waiting on 'niche' info from Arielli
tem_filtered = tem[['Rotated Lat', 'Rotated Lon', 'genus', 'taxon_environment', 
                    'age_max', 'age_min']]
tem_filtered.isna().sum()

Rotated Lat           0
Rotated Lon           0
genus                 0
taxon_environment    14
age_max               0
age_min               0
dtype: int64

In [18]:
# Still waiting on 'niche' info from Arielli
rep_filtered = rep[['Rotated Lat', 'Rotated Lon', 'genus', 'taxon_environment', 
                    'age_max', 'age_min']]
rep_filtered.isna().sum()

Rotated Lat          0
Rotated Lon          0
genus                0
taxon_environment    0
age_max              0
age_min              0
dtype: int64

In [41]:
rep_filtered['taxon_environment'].head(20)

0     terrestrial
1     terrestrial
2     terrestrial
3     terrestrial
4     terrestrial
5     terrestrial
6     terrestrial
7     terrestrial
8     terrestrial
9     terrestrial
10    terrestrial
11    terrestrial
12    terrestrial
13    terrestrial
14    terrestrial
15    terrestrial
16    terrestrial
17    terrestrial
18    terrestrial
19    terrestrial
Name: taxon_environment, dtype: object

In [42]:
rep['environment'].head(20)

0     lacustrine - large
1     lacustrine - large
2           fissure fill
3           fissure fill
4         fluvial indet.
5     terrestrial indet.
6     terrestrial indet.
7           fissure fill
8           fissure fill
9           fissure fill
10          fissure fill
11          fissure fill
12          fissure fill
13          fissure fill
14          fissure fill
15          fissure fill
16          fissure fill
17          fissure fill
18          fissure fill
19          fissure fill
Name: environment, dtype: object

In [92]:
rep_filtered.to_csv('reptilia_processed_data/reptilia_bdnn.csv')
syn_filtered.to_csv('synapsida_processed_data/synapsida_bdnn.csv')
tem_filtered.to_csv('temnospondyli_processed_data/temnospondyli_bdnn.csv')


## Prepping DeepDive columns

In [93]:
rep_deepdive = rep[['genus', 'county', 'state', 'occurrence_no']]
rep_deepdive.head()

,genus,county,state,occurrence_no
0,Icarosaurus,Hudson,New Jersey,146084
1,Rutiodon,Hudson,New Jersey,146085
2,Kuehneosuchus,Somerset,England,146086
3,Kuehneosaurus,Somerset,England,146087
4,Procolophon,Chris Hani,Eastern Cape,147521


In [94]:
syn_deepdive = syn[['genus', 'county', 'state', 'occurrence_no']]
tem_deepdive = tem[['genus', 'county', 'state', 'occurrence_no']]

### 'County' and 'State' Cols EDA

#### Are 'County' Null Counts Reasonable?

In [87]:
rep_deepdive.isna().sum()

# 'county' has a lot of nulls, so I'm going to look back at the older datasets and see if that seems correct or
# if an error occurred when I was duplicating the rows

genus               0
county           1495
state             212
occurrence_no       0
dtype: int64

In [88]:
# Making sure occurrence no has >1 values, since there are duplicate rows (duplicated in eda_duplicates.ipynb)
rep['occurrence_no'].value_counts()

1285964    300
629477      91
1586159     37
902070      33
486411      25
          ... 
830763       1
830777       1
830855       1
830859       1
N84          1
Name: occurrence_no, Length: 3145, dtype: int64

In [24]:
# Loading in my original duplication output
rep_original = pd.read_csv('occurrence_tables_1/reptilia_dup.csv')
rep_original_filtered = rep_original[['genus', 'county', 'state']]
rep_original_filtered.isna().sum()

genus        0
county    1495
state      212
dtype: int64

In [27]:
# Loading in the very first dataset (raw, no rotated lat and long)
rep_old = pd.read_csv('occurrence_tables_1/reptilia_genus_occ_all.csv', encoding = 'latin1')
rep_old_filtered = rep_old[['genus', 'county', 'state']]
rep_old_filtered.isna().sum()

genus       0
county    920
state     197
dtype: int64

#### Subsetting and Saving Just Duplicated Rows

In [95]:
# Saving off just the parts in the deepdive dataset that were duplicated occurrences (i.e., the rows that were duplicated in eda_duplicates.ipynb)
# Trying to see if there's a pattern in the 'county' nulls that I can fix
rep_deepdive_dups = rep_deepdive[rep_deepdive.duplicated(subset=['occurrence_no'], keep=False)]
rep_deepdive_dups.to_csv('reptilia_processed_data/reptilia_deepdive_dups.csv')

syn_deepdive_dups = syn_deepdive[syn_deepdive.duplicated(subset=['occurrence_no'], keep=False)]
syn_deepdive_dups.to_csv('synapsida_processed_data/synapsida_deepdive_dups.csv')

tem_deepdive_dups = tem_deepdive[tem_deepdive.duplicated(subset=['occurrence_no'], keep=False)]
tem_deepdive_dups.to_csv('temnospondyli_processed_data/temnospondyli_deepdive_dups.csv')

#### Fixing Nonsensical Strings in 'County' and 'State'

In [108]:
# Return rows in 'county' and 'state' columns that contain any non-letter characters
import re

# Function for outputting a CSV of unique values in a column that contain special characters
def find_special_chars(df, col, clade):
    df2 = df.copy()
    df2[col] = df2[col].fillna('') # Have to fill empty values with something to use regex
    non_letter = df2[df2[col].str.contains(r'[^a-zA-Z\s\-/]', na=False)] # Regex for non-letter characters, excluding hyphens and forward slashes
    unique = non_letter[col].unique()
    # Saving unique values to a CSV named after the clade and column
    unique_df = pd.DataFrame(unique)
    unique_df.to_csv(f'{clade}_processed_data/{clade}_{col}_special_chars.csv') 
    return unique

In [114]:
# Reptilia
find_special_chars(rep_deepdive, 'county', 'reptilia'), find_special_chars(rep_deepdive, 'state', 'reptilia')

(array(['Valle Fert√É¬≠l', 'Valle F√É¬©rtil', 'Vend√É¬©e',
        'Wei√É¬üenburg-Gunzenhausen', 'S√É¬£o Gabriel', "Mezen'",
        'L√É¬∂rrach', 'Portel√É¬¢ndia', 'Perol√É¬¢ndia',
        'S√É¬£o Mateus do Sul', 'Tatu√É¬≠', 'Rebou√É¬ßas', 'Candel√É¬°ria',
        "Sol'-Iletsk", "Sol'-Iletskiy", 'N√É¬ºrnberger Land', "Yuan'an",
        'Schw√É¬§bisch Hall', 'S√É¬£o Pedro do Sul', 'G√É¬∂ttingen',
        'Sa√É¬¥ne-et-Loire', 'Baden-W√É¬ºrttemberg', 'Th√É¬ºringen',
        'T√É¬ºbingen', 'S√É¬¢one-et-Loire', 'H√É¬©rault',
        'S√É¬£o Jo√É¬£o do Pol√É¬™sine', 'Yuan√¢¬Ä¬ôan',
        'Tarnowskie G√É¬≥ry', 'Los Colorados, La Esquina', 'Arroio do S¬õ',
        'Linha S?o Luiz', 'Toroqu√ø', 'Ribeir?o', 'Rinc?o dos Weiss',
        'Acegu√ø', 'Sesmaria do Pinhal 1',
        'cff Rafael prov√øvel material do MNRJ se sao os mesmo do livro tombo, Alemoa',
        'Cerrito?', 'Sesmaria do Pinhal 2', 'V√ørzea do Agudo',
        '?gua Negra', 'BR 158 federal road', 'Porto Mariante 2',
        'S

In [116]:
# Temnospondyli
find_special_chars(tem_deepdive, 'county', 'temnospondyli'), find_special_chars(tem_deepdive, 'state', 'temnospondyli')

(array(["Sol'-Iletsk", 'Wei√É¬üeritzkreis', "Sol'-Iletskiy",
        'Schw√É¬§bisch Hall', 'T√É¬ºbingen', 'Sa√É¬¥ne-et-Loire',
        'S√É¬£o Jo√É¬£o do Pol√É¬™sine', 'S√É¬£o Gabriel', 'Sieker,',
        'Valle F√É¬©rtil', 'S?o Jer¬ìnimo da Serra', 'Toroqu¬†'],
       dtype=object),
 array(["Archangel'sk", 'Baden-W√É¬ºrttemberg', 'Sk√É¬•ne', 'Th√É¬ºringen',
        'Azad Jammu & Kashmir', 'Rakovn√É¬≠k', 'Th√É¬ºringer Wald',
        'Th√É¬ºringia', 'Paran√É¬°', "Arkhangel'sk", 'Baden-W√É¬ºrtteberg'],
       dtype=object))

In [115]:
# Synapsida
find_special_chars(syn_deepdive, 'county', 'synapsida'), find_special_chars(syn_deepdive, 'state', 'synapsida')

(array(["Sol'-Iletsk", 'Valle Fert√É¬≠l', 'Kodzko (Glatz, Kladsko)',
        'Sa√É¬¥ne-et-Loire', 'Wei√É¬üeritzkreis', 'Victoria West (Ubuntu)',
        "Mezen'", "Sol'-Iletskiy", 'Schw√É¬§bisch Hall', 'S√É¬£o Gabriel',
        'Candel√É¬°ria', 'Kotel√¢¬Ä¬ônichskii', 'T√É¬ºbingen',
        '√É¬úr√É¬ºmqi', "Zav'yalovskii", "Afanas'evskii", "Sarapul'skii",
        "Oktyabr'skii", "Tyul'ganskii", 'H√É¬©rault', 'Valle F√É¬©rtil',
        'Vila Est¬Éncia Nova', 'Linha S?o Luiz', 'Linha S?o Luiz ',
        'Sesmaria do Pinhal 1', 'V¬†rzea do Agudo', 'Botucara¬°',
        'Rinc?o do Pinhal', 'Linha V¬†rzea 2', 'Linha V¬†rzea 1',
        'Rinc?o do Semi?o ', 'Porto Mariante 2', 'Rinc?o da Porta',
        'Catu¬áaba', 'BR 158 federal road', 'Linha Fac?o', 'Chiniqu¬†'],
       dtype=object),
 array(['Franch-Comt√É¬©', "Mohale's Hoek", 'R√É¬≠o Negro',
        'Baden-W√É¬ºrttemberg', 'Rakovn√É¬≠k', 'Paran√É¬°', 'H√É¬©rault'],
       dtype=object))

#### Imputing 'County' Nulls

In [ ]:
rep_deepdive['county'] = rep_deepdive.apply(lambda row: row['state'] if row['county'] == '-' else row['county'], axis=1)

rep_deepdive.loc[rep_deepdive['county'] == '-'] = rep_deepdive['state']

In [55]:
len(rep_deepdive[rep_deepdive['county'] == "-"])

43

In [56]:
rep_deepdive[rep_deepdive['county'] == "-"]

,id,genus,county,state,occurrence_no
4245,166,Pareiasaurus,-,ECAPE,N100
4246,165,Erythrosuchus,-,ECAPE,N101
4247,164,Erythrosuchus,-,ECAPE,N102
4248,163,Erythrosuchus,-,ECAPE,N103
4249,162,Pareiasuchus,-,WCAPE,N104
4250,161,Eohyosaurus,-,FSTATE,N105
4251,160,Erythrosuchus,-,WCAPE,N106
4252,159,Erythrosuchus,-,ECAPE,N107
4253,158,Erythrosuchus,-,WCAPE,N108
4254,157,Erythrosuchus,-,WCAPE,N109
